# Elo Ranking

In this notebook, we will calculate the Elo ranking based on the user votes.

## Load votes

In [1]:
# Load all votes
import pandas as pd

votesDF = pd.read_csv("data/votes.csv", sep=";", names=["id","timestamp","user","id_context","id_plan_win","id_plan_lost","dislikes"], index_col=False, parse_dates=[1])
print("Total votes: " + str(len(votesDF)))

Total votes: 1705


## Votes stadistics

We will analyze the votes. Firstly, we show how many users voted.

In [2]:
usersDF = votesDF.groupby(by='user').count()
print("Total users: " + str(len(usersDF)))

Total users: 71


In [3]:
votesDF.head()

,id,timestamp,user,id_context,id_plan_win,id_plan_lost,dislikes
0,0,2018-01-18 10:52:59.233,user-0,20,4,8,NaN
1,1,2018-01-18 10:53:54.404,user-0,29,3,8,NaN
2,2,2018-01-18 10:54:34.714,user-0,29,0,1,NaN
3,3,2018-01-18 10:55:21.262,user-0,29,9,6,NaN
4,4,2018-01-18 10:56:06.951,user-0,22,8,7,NaN


Next, we will calculate the time spend between 2 votes from the same user.

In [4]:
votesDF['timelapse'] = votesDF.timestamp - votesDF.timestamp.shift(1)
votesDF['timelapse'] = votesDF['timelapse'].map(lambda x: x.total_seconds())

usersStats = votesDF[['user', 'timelapse']].groupby(by='user').agg({'timelapse':['mean', 'min', 'max']})
usersStats.head()

timelapse                   
               mean      min       max
user                                  
user-0    89.771125   30.118   409.745
user-1   265.774500   17.864   985.754
user-10   72.107800   11.100   209.710
user-11  139.089226    0.302  3244.703
user-12  751.412500  184.316  1318.509

We will filter all votes with a timelapse less than 5 seconds

In [5]:
filteredFastVotes = votesDF[votesDF.timelapse >= 5]
print("Total filtered votes: " + str(len(filteredFastVotes)))

Total filtered votes: 814


## Plan stadistics

Firstly, we will apply a new id for each case. In our case, this id is: `C + id. context + - + id. plan`. In `userStats` we will add 2 columns, the winner and lost plan in each vote.

In [6]:
filteredFastVotes['id_win'] = filteredFastVotes.apply(lambda row: 'C{}-{}'.format(str(row.id_context), str(row.id_plan_win)), axis=1)
filteredFastVotes['id_lost'] = filteredFastVotes.apply(lambda row: 'C{}-{}'.format(str(row.id_context), str(row.id_plan_lost)), axis=1)

/Users/jljorro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/jljorro/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
filteredFastVotes.head()

,id,timestamp,user,id_context,id_plan_win,id_plan_lost,dislikes,timelapse,id_win,id_lost
1,1,2018-01-18 10:53:54.404,user-0,29,3,8,NaN,55.171,C29-3,C29-8
2,2,2018-01-18 10:54:34.714,user-0,29,0,1,NaN,40.310,C29-0,C29-1
3,3,2018-01-18 10:55:21.262,user-0,29,9,6,NaN,46.548,C29-9,C29-6
4,4,2018-01-18 10:56:06.951,user-0,22,8,7,NaN,45.689,C22-8,C22-7
5,5,2018-01-18 11:02:56.696,user-0,5,1,5,NaN,409.745,C5-1,C5-5


Next, we will order each case based on the number of votes.

In [8]:
votedPlans = pd.DataFrame(data=filteredFastVotes[['timestamp','id_win']].values, columns=['timestamp', 'id'])
votedPlans['Result'] = "Win"

lostPlan = pd.DataFrame(data=filteredFastVotes[['timestamp','id_lost']].values, columns=['timestamp', 'id'])
lostPlan['Result'] = 'Lost'

votedPlans = votedPlans.append(lostPlan, ignore_index=True)

planStats = votedPlans.groupby(by=['id']).count()
planStats.reset_index(inplace=True)

In [9]:
planStats.sort_values('Result', ascending=False, inplace=True)

import plotly
from plotly.graph_objs import *

plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [Bar(x=planStats.id, y=planStats.Result)],
    "layout": Layout(title="Cases order by the number of votes")
})


Next, we will show the votes ordered by the number of winner votes and lost votes.

In [10]:
planStatsByResult = votedPlans.groupby(by=['id','Result']).count()
planStatsByResult.reset_index(inplace=True)
planStatsByResult.head()

planStatsByResult.sort_values('timestamp', ascending=False, inplace=True)

wins = Bar(x=planStatsByResult[planStatsByResult.Result == "Win"].id, y = planStatsByResult[planStatsByResult.Result == "Win"].timestamp, name = "Winner")
losts = Bar(x=planStatsByResult[planStatsByResult.Result == "Lost"].id, y = planStatsByResult[planStatsByResult.Result == "Lost"].timestamp, name="Looser")

data = [wins, losts]
layout = Layout(
    barmode='stack',
    title = "Plan stats by result"
)



plotly.offline.iplot({
    "data": data,
    "layout": layout
})
        

## Elo ranking

Next, we will calculate the Elo ranking using the valid votes. We only consider cases with 4 votes or more.

In [11]:
invalidPlansId = planStats[planStats.Result < 4].id.unique()
len(invalidPlansId)

63

In [12]:
validPlans = filteredFastVotes[~filteredFastVotes.id_win.isin(invalidPlansId)]
validPlans = validPlans[~validPlans.id_lost.isin(invalidPlansId)]
validPlans.head()

,id,timestamp,user,id_context,id_plan_win,id_plan_lost,dislikes,timelapse,id_win,id_lost
1,1,2018-01-18 10:53:54.404,user-0,29,3,8,NaN,55.171,C29-3,C29-8
2,2,2018-01-18 10:54:34.714,user-0,29,0,1,NaN,40.310,C29-0,C29-1
3,3,2018-01-18 10:55:21.262,user-0,29,9,6,NaN,46.548,C29-9,C29-6
4,4,2018-01-18 10:56:06.951,user-0,22,8,7,NaN,45.689,C22-8,C22-7
6,6,2018-01-18 11:03:44.564,user-0,20,9,0,NaN,47.868,C20-9,C20-0


In [13]:
# Init Elo Rating
init_elo_value = 0.0

rankingElo = pd.DataFrame(planStats[planStats.Result >= 4].id.unique(), columns=['id'])
rankingElo['elo_rating'] = init_elo_value
rankingElo.head()

,id,elo_rating
0,C29-8,0.0
1,C29-7,0.0
2,C29-9,0.0
3,C29-3,0.0
4,C29-2,0.0


In [14]:
import math

def estimate(points_a, points_b):
    return 1.0 / (1.0 + math.pow(10.0, (points_b - points_a) / 400.0))

def recalculate_points(points, estimation, result):
    const_elo = 30.0
    return points + const_elo * (result - estimation)

In [15]:
for index, row in validPlans.iterrows():
    
    win_points = rankingElo[rankingElo.id == row['id_win']]['elo_rating'].values[0]
    lost_points = rankingElo[rankingElo.id == row['id_lost']]['elo_rating'].values[0]
    
    # Estimación del ambos planes
    estimation_win = estimate(win_points, lost_points)
    estimation_lost = estimate(lost_points, win_points)
    
    new_win_points = recalculate_points(win_points, estimation_win, 1.0)
    new_lost_points = recalculate_points(lost_points, estimation_lost, 0.0)
    
    rankingElo.loc[rankingElo.id == row['id_win'], 'elo_rating'] = new_win_points
    rankingElo.loc[rankingElo.id == row['id_lost'], 'elo_rating'] = new_lost_points

rankingElo.sort_values('elo_rating', ascending=False, inplace=True)
rankingElo.head()
len(rankingElo[rankingElo.elo_rating > 0])

113

In [16]:
layout = Layout(
    xaxis=dict(title='Cases Id'),
    yaxis=dict(title='Elo points')
);

plotly.offline.iplot({
    "data": [Bar(x=rankingElo.id, y=rankingElo.elo_rating)],
    "layout": layout
})